# Computation

In [1]:
#import libraries cell

import json
import time
import findspark
import pyspark

import numpy as np
import pandas as pd

from kafka import KafkaProducer
from kafka.admin import KafkaAdminClient, NewTopic
 
from pyspark.sql           import functions as F
from pyspark.sql           import SparkSession
from pyspark.streaming     import StreamingContext
from pyspark.sql.types     import StructField, StructType, DoubleType, IntegerType
from pyspark.sql.functions import from_json, to_json, col, when, sum, count, struct, collect_list
from pyspark               import SparkConf, SparkContext

## Spark setup

In [2]:
#initialisation of spark from the packages folder
findspark.init('/usr/local/spark')

In [3]:
#start session - specify port, application name, and configuration settings.

spark = SparkSession.builder \
        .appName("Project_CosmicRays_Dashboard_application")\
        .config("spark.sql.execution.arrow.pyspark.enabled", "true")\
        .config("spark.sql.execution.arrow.pyspark.fallback.enabled", "false")\
        .config("spark.sql.streaming.forceDeleteTempCheckpointLocation", "true")\
        .config("spark.jars.packages","org.apache.spark:spark-sql-kafka-0-10_2.12:3.1.2")\
        .config("spark.ui.port", "4041")\
        .getOrCreate()

sc = spark.sparkContext
spark

:: loading settings :: url = jar:file:/usr/local/spark/jars/ivy-2.5.0.jar!/org/apache/ivy/core/settings/ivysettings.xml


Ivy Default Cache set to: /home/aidin/.ivy2/cache
The jars for the packages stored in: /home/aidin/.ivy2/jars
org.apache.spark#spark-sql-kafka-0-10_2.12 added as a dependency
:: resolving dependencies :: org.apache.spark#spark-submit-parent-c2a6b55a-b4b6-4cf2-908b-d8fd13936c9c;1.0
	confs: [default]
	found org.apache.spark#spark-sql-kafka-0-10_2.12;3.1.2 in central
	found org.apache.spark#spark-token-provider-kafka-0-10_2.12;3.1.2 in central
	found org.apache.kafka#kafka-clients;2.6.0 in central
	found com.github.luben#zstd-jni;1.4.8-1 in central
	found org.lz4#lz4-java;1.7.1 in central
	found org.xerial.snappy#snappy-java;1.1.8.2 in central
	found org.slf4j#slf4j-api;1.7.30 in central
	found org.spark-project.spark#unused;1.0.0 in central
	found org.apache.commons#commons-pool2;2.6.2 in central
:: resolution report :: resolve 725ms :: artifacts dl 60ms
	:: modules in use:
	com.github.luben#zstd-jni;1.4.8-1 from central in [default]
	org.apache.commons#commons-pool2;2.6.2 from central i

22/09/13 19:53:14 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable


Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).


## Kafka Setup

In [4]:
KAFKA_BOOTSTRAP_SERVERS = 'localhost:9092'

producer = KafkaProducer(bootstrap_servers=KAFKA_BOOTSTRAP_SERVERS)

In [5]:
kafka_admin = KafkaAdminClient(bootstrap_servers=KAFKA_BOOTSTRAP_SERVERS)

In [6]:
kafka_admin.list_topics()

['stream', 'results', '__consumer_offsets']

In [7]:
kafka_admin.delete_topics(['results'])


DeleteTopicsResponse_v3(throttle_time_ms=0, topic_error_codes=[(topic='results', error_code=0)])

In [8]:
kafka_admin.list_topics()

['stream', '__consumer_offsets']

In [12]:
results_topic = NewTopic(name='results', 
                       num_partitions=4, 
                       replication_factor=1)
kafka_admin.create_topics(new_topics=[results_topic])


CreateTopicsResponse_v3(throttle_time_ms=0, topic_errors=[(topic='results', error_code=0, error_message=None)])

In [13]:
kafka_admin.list_topics()

['stream', 'results', '__consumer_offsets']

In [14]:
#define the input dataframe and its source. Define subscription to 'stream' - one of the two topics in kafka
inputDF = spark\
        .readStream\
        .format("kafka")\
        .option("kafka.bootstrap.servers", KAFKA_BOOTSTRAP_SERVERS)\
        .option('subscribe', 'stream')\
        .load()

inputDF.printSchema()

root
 |-- key: binary (nullable = true)
 |-- value: binary (nullable = true)
 |-- topic: string (nullable = true)
 |-- partition: integer (nullable = true)
 |-- offset: long (nullable = true)
 |-- timestamp: timestamp (nullable = true)
 |-- timestampType: integer (nullable = true)



In [15]:
#define the schema of the rows that will be read. double are used to overcome overflow issues
schema = StructType(
        [
            StructField("HEAD",        IntegerType()),
            StructField("FPGA",        IntegerType()),
            StructField("TDC_CHANNEL", IntegerType()),
            StructField("ORBIT_CNT",    DoubleType()),
            StructField("BX_COUNTER",   DoubleType()), 
            StructField("TDC_MEAS",     DoubleType())
        ]
    )

In [16]:
#convert input_Df to json by casting columns into the predefined schema.
jsonDF = inputDF.select(from_json(col("value").alias('value').cast("string"), schema).alias('value'))

jsonDF.printSchema()

root
 |-- value: struct (nullable = true)
 |    |-- HEAD: integer (nullable = true)
 |    |-- FPGA: integer (nullable = true)
 |    |-- TDC_CHANNEL: integer (nullable = true)
 |    |-- ORBIT_CNT: double (nullable = true)
 |    |-- BX_COUNTER: double (nullable = true)
 |    |-- TDC_MEAS: double (nullable = true)



In [17]:
#flattening the dataframe
flatDF = jsonDF.selectExpr("value.HEAD", 
                           "value.FPGA", 
                           "value.TDC_CHANNEL",
                           "value.ORBIT_CNT",
                           "value.BX_COUNTER",
                           "value.TDC_MEAS")

flatDF.printSchema()

root
 |-- HEAD: integer (nullable = true)
 |-- FPGA: integer (nullable = true)
 |-- TDC_CHANNEL: integer (nullable = true)
 |-- ORBIT_CNT: double (nullable = true)
 |-- BX_COUNTER: double (nullable = true)
 |-- TDC_MEAS: double (nullable = true)



In [23]:
# data-cleansing, removing ancillary hits
# division of dataframe between chambers
# adding absolute time column to define DRIFTIME
df = flatDF.filter(col('HEAD') == 2)\
    .withColumn('CHAMBER',
                when(( col( 'FPGA' ) == 0 ) & ( col( 'TDC_CHANNEL' ) <= 63 ),                                   0) \
               .when(( col( 'FPGA' ) == 0 ) & ( col( 'TDC_CHANNEL' ) >  63 ) & ( col( 'TDC_CHANNEL' ) <= 127 ), 1) \
               .when(( col( 'FPGA' ) == 1 ) & ( col( 'TDC_CHANNEL' ) <= 63 ),                                   2) \
               .when(( col( 'FPGA' ) == 1 ) & ( col( 'TDC_CHANNEL' ) >  63 ) & ( col( 'TDC_CHANNEL' ) <= 127 ), 3) \
               .when(( col( 'FPGA' ) == 1 ) &                                  ( col( 'TDC_CHANNEL' ) == 128 ) ,4) \
               .otherwise(None))                                                                                   \
    .filter( col( 'CHAMBER' ).isNotNull() )                                                                        \
    .withColumn('ABSOLUTETIME', 25 * ( col( 'ORBIT_CNT' ) * 3564 + col( 'BX_COUNTER' ) + col( 'TDC_MEAS' ) / 30 ) )

df.printSchema()

root
 |-- HEAD: integer (nullable = true)
 |-- FPGA: integer (nullable = true)
 |-- TDC_CHANNEL: integer (nullable = true)
 |-- ORBIT_CNT: double (nullable = true)
 |-- BX_COUNTER: double (nullable = true)
 |-- TDC_MEAS: double (nullable = true)
 |-- CHAMBER: integer (nullable = true)
 |-- ABSOLUTETIME: double (nullable = true)



In [19]:
#scintillator time offset by Chamber
time_offset_by_chamber = {
0: 95.0 - 1.1, # Ch 0
1: 95.0 + 6.4, # Ch 1
2: 95.0 + 0.5, # Ch 2
3: 95.0 - 2.6, # Ch 3
}

In [31]:
msg_json = {
    'epoch_id' : 0,
    'hits'     : 0,
    'CH0' : {'total_hits'  : 0,
             'histo_CH'    : {'bin_edges'  : [],
                              'bin_counts' : []
                            },
             'histo_ORB'   : {'bin_edges'  : [],
                              'bin_counts' : []
                             },
             'histo_SC'    : {'bin_edges'  : [],
                              'bin_counts' : []
                            },
             'DRIFTIME'    : []
            },
    'CH1' : {'total_hits'  : 0,
             'histo_CH'    : {'bin_edges'  : [],
                              'bin_counts' : []
                            },
             'histo_ORB'   : {'bin_edges'  : [],
                              'bin_counts' : []
                             },
             'histo_SC'    : {'bin_edges'  : [],
                              'bin_counts' : []
                            },
             'DRIFTIME'    : []
            },
    'CH2' : {'total_hits'  : 0,
             'histo_CH'    : {'bin_edges'  : [],
                              'bin_counts' : []
                            },
             'histo_ORB'   : {'bin_edges'  : [],
                              'bin_counts' : []
                             },
             'histo_SC'    : {'bin_edges'  : [],
                              'bin_counts' : []
                            },
             'DRIFTIME'    : []
            },
    'CH3' : {'total_hits'  : 0,
             'histo_CH'    : {'bin_edges'  : [],
                              'bin_counts' : []
                            },
             'histo_ORB'   : {'bin_edges'  : [],
                              'bin_counts' : []
                             },
             'histo_SC'    : {'bin_edges'  : [],
                              'bin_counts' : []
                            },
             'DRIFTIME'    : []
            },
}

In [32]:
def batch_proc(df, epoch_id):   

    '''
    Function for the batch processing of the data.
    The processing consists in retrieving
    the following informations:
        1. total number of processed hits, 
           post-clensing (1 value per batch)
        2. total number of processed hits,
           post-clensing, per chamber (4 values per batch)
        3. histogram of the counts of active TDC_CHANNEL,
           per chamber (4 arrays per batch)
        4. histogram of the total number of active
           TDC_CHANNEL in each ORBIT_CNT, per chamber 
           (4 arrays per batch)
        5. histogram of the counts of active TDC_CHANNEL,
           per chamber, ONLY for those orbits with at least
           one scintillator signal in it (4 arrays per batch)
        6. histogram of the DRIFTIME, per chamber 
           (4 arrays per batch)
    
    Inputs:
        - df: spark dataframe with the data
        - epoch_id: batch index    
    '''

    msg_json['CH0']['hits'] = 0
    msg_json['CH1']['hits'] = 0
    msg_json['CH2']['hits'] = 0
    msg_json['CH3']['hits'] = 0
    msg_json['epoch_id']    = epoch_id


    # total number of processed hits
    # post-cleansing
    total_hits = df.filter(col('CHAMBER') != 4)\
                   .count()
    

    # total number of processed hits,
    # post-clensing, per chamber
    df_counts = df.filter(col('CHAMBER') != 4) \
                  .groupBy(   'CHAMBER')       \
                  .count()                     \
                  .withColumnRenamed('count', 'ch_hits')


    # histogram of the counts of active
    # TDC_CHANNEL, per chamber.
    df_counts_ch = df.filter(col('CHAMBER') != 4)            \
                     .groupBy('TDC_CHANNEL', 'CHAMBER')      \
                     .count()                                \
                     .groupBy('CHAMBER')                     \
                     .agg(struct(collect_list('TDC_CHANNEL') \
                                       .alias('bin_edges'),
                                 collect_list(    'count')   \
                                       .alias(   'counts'),
                                )                            \
                          .alias('histo_CH')
                         )
    
    
    # histogram of the total number of active
    # TDC_CHANNEL in each ORBIT_CNT per chamber
    
    df_counts_orb = df.filter(col('CHAMBER') != 4)            \
                      .groupBy('ORBIT_CNT', 'CHAMBER')        \
                      .agg(F.countDistinct('TDC_CHANNEL'))    \
                      .groupBy(col('count(TDC_CHANNEL)'))     \
                      .count()                                \
                      .groupBy('CHAMBER')                     \
                      .agg(struct(collect_list('TDC_CHANNEL') \
                                        .alias(  'bin_edges'),
                                  collect_list(      'count') \
                                        .alias(     'counts'),
                                 )                            \
                           .alias('histo_ORB')
                          )


    # Histogram of the count of active TDC_CHANNEL,
    # per chamber, only for those orbits with
    # at least one scintillatorin it

    df_scint = df.select('CHAMBER', 'ORBIT_CNT', 'ABSOLUTETIME') \
                     .filter(col('CHAMBER') == 4)                    \
                     .groupBy('ORBIT_CNT')                           \
                     .min('ABSOLUTETIME')                            \
                     .withColumnRenamed('min(ABSOLUTETIME)', 't0')
    list_scint = df_sc['ORBIT_CNT'].tolist()
    
    
    df_sc = df.filter(col('CHAMBER') != 4 )                   \
              .where( col( 'ORBIT_CNT' ).isin( list_scint ) ) \
              .groupBy(  'TDC_CHANNEL', 'CHAMBER'       )     \
              .count()                                        \
              .agg(struct(collect_list('TDC_CHANNEL')         \
                                  .alias('bin_edges'),
                            collect_list(    'count')         \
                                  .alias(   'counts'),
                         ).alias('histo_SC')
                  )
 
    
    # histogram of the DRIFTIME, per chamber (4 arrays per batch)
    df_drift = df.select('ORBIT_CNT', 'CHAMBER', 'ABSOLUTETIME') \
                 .filter(col('chamber') != 4)                  \
                 .join(df_scint, on='ORBIT_CNT', how='rightouter')\
                 .withColumn('DRIFTIME',
                           when(col('CHAMBER') == 0, col('ABSOLUTETIME') - (col('t0') - time_offset_by_chamber[0]))
                          .when(col('CHAMBER') == 1, col('ABSOLUTETIME') - (col('t0') - time_offset_by_chamber[1]))
                          .when(col('CHAMBER') == 2, col('ABSOLUTETIME') - (col('t0') - time_offset_by_chamber[2]))
                          .when(col('CHAMBER') == 3, col('ABSOLUTETIME') - (col('t0') - time_offset_by_chamber[3]))
                           )\
                .groupBy('CHAMBER')\
                .agg(collect_list('DRIFTIME').alias('DRIFTIME'))

    msg_json['hits'] = total_hits
    for i, ch in enumerate(df_counts['CHAMBER']):
        msg_json[f'CH{chamber}']['total_hits']             = df_counts[    ch]['ch_hits']
        msg_json[f'CH{chamber}']['hist_CH'][  'bin_edges'] = df_counts_ch[ ch]['hist_CHANNEL']['bin_edges']
        msg_json[f'CH{chamber}']['hist_CH'][' bin_counts'] = df_counts_ch[ ch]['hist_CHANNEL']['counts']
        msg_json[f'CH{chamber}']['hist_ORB'][ 'bin_edges'] = df_counts_orb[ch]['hist_ORBIT']['bin_edges']
        msg_json[f'CH{chamber}']['hist_ORB']['bin_counts'] = df_counts_orb[ch]['hist_ORBIT']['counts']
        msg_json[f'CH{chamber}']['hist_SC'][  'bin_edges'] = df_sc[        ch]['hist_SCINT']['bin_edges']
        msg_json[f'CH{chamber}']['hist_SC'][ 'bin_counts'] = df_sc[        ch]['hist_SCINT']['counts']
        msg_json[f'CH{chamber}']['DRIFTIME']               = df_drift[     ch]['DRIFTIME']

    producer.send('results', json.dumps(msg_json).encode('utf-8'))
    producer.flush()
    pass

In [33]:
flatDF.isStreaming

True

In [34]:
flatDF.writeStream\
    .foreachBatch(batch_proc)\
    .start()\
    .awaitTermination()

22/09/13 22:41:54 WARN ResolveWriteToStream: Temporary checkpoint location created which is deleted normally when the query didn't fail: /tmp/temporary-cadfb525-7e2e-4e4f-ac13-9545bb7349ce. If it's required to delete it under any circumstances, please set spark.sql.streaming.forceDeleteTempCheckpointLocation to true. Important to know deleting temp checkpoint folder is best effort.
22/09/13 22:41:54 WARN ResolveWriteToStream: spark.sql.adaptive.enabled is not supported in streaming DataFrames/Datasets and will be disabled.
22/09/13 22:41:54 ERROR MicroBatchExecution: Query [id = 90d65950-a64b-4020-96e4-72252a501ee6, runId = 6424efc8-399d-4bd5-a802-fc97391c3f67] terminated with error
py4j.Py4JException: An exception was raised by the Python Proxy. Return Message: Traceback (most recent call last):
  File "/home/aidin/.local/lib/python3.9/site-packages/py4j/clientserver.py", line 617, in _call_proxy
    return_value = getattr(self.pool[obj_id], method)(*params)
  File "/home/aidin/.local

StreamingQueryException: Query [id = 90d65950-a64b-4020-96e4-72252a501ee6, runId = 6424efc8-399d-4bd5-a802-fc97391c3f67] terminated with exception: An exception was raised by the Python Proxy. Return Message: Traceback (most recent call last):
  File "/home/aidin/.local/lib/python3.9/site-packages/py4j/clientserver.py", line 617, in _call_proxy
    return_value = getattr(self.pool[obj_id], method)(*params)
  File "/home/aidin/.local/lib/python3.9/site-packages/pyspark/sql/utils.py", line 272, in call
    raise e
  File "/home/aidin/.local/lib/python3.9/site-packages/pyspark/sql/utils.py", line 269, in call
    self.func(DataFrame(jdf, self.session), batch_id)
  File "/tmp/ipykernel_4533/621440559.py", line 36, in batch_proc
    total_hits = df.filter(col('CHAMBER') != 4)\
  File "/home/aidin/.local/lib/python3.9/site-packages/pyspark/sql/dataframe.py", line 2079, in filter
    jdf = self._jdf.filter(condition._jc)
  File "/home/aidin/.local/lib/python3.9/site-packages/py4j/java_gateway

In [ ]:
spark.stop()